In [ ]:
import os
from keras.models import Sequential
from keras.callbacks import LambdaCallback, ModelCheckpoint
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt
import random
import sys
import io

In [ ]:
# Text processing

path = '../input/green.txt' 
with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [ ]:
# Single LSTM layer model - Not much dropout
print('Build model...')
model_single = Sequential()
model_single.add(LSTM(128, input_shape=(maxlen, len(chars))))
model_single.add(Dropout(.3))
model_single.add(Dense(len(chars), activation='softmax'))
model_single.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.01))

In [ ]:
# Two layer LSTM - More dropout.
print('Build model...')
model_double = Sequential()
model_double.add(LSTM(128, input_shape=(maxlen, len(chars)),return_sequences=True))
model_double.add(Dropout(.5))
model_double.add(LSTM(128))
model_double.add(Dropout(.5))
model_double.add(Dense(len(chars), activation='softmax'))
model_double.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.01))

In [ ]:
models = {'single': model_single, 'double': model_double}

In [ ]:
filepath_single = "single_weights.hdf5"
filepath_double= "double_weights.hdf5"
checkpoint_single = ModelCheckpoint(filepath_single, 
                             monitor='val_loss', 
                             verbose=1, 
                             save_best_only=True, 
                             mode='min')
checkpoint_double = ModelCheckpoint(filepath_double, 
                             monitor='val_loss', 
                             verbose=1, 
                             save_best_only=True, 
                             mode='min')
checkpoints= {'single': checkpoint_single, 'double': checkpoint_double}

In [ ]:
# Text Generation Functions

def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def generate_text(model):
    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

In [ ]:
history = dict()
for name, model in models.items():
    history[name] = model.fit(x, y,
                              batch_size=2048,
                              epochs=40,
                              validation_split=.15,
                              verbose=2,
                              shuffle=False,
                              callbacks=[checkpoints[name]])

In [ ]:
for name in models.keys():
    plt.plot(history[name].history['loss'])
    plt.plot(history[name].history['val_loss'])
    plt.title('Training History')
    plt.ylabel('loss')
    plt.xlabel('epoch')
plt.legend(['train-single', 'val-single','train-double', 'val-double'], loc='upper right')
plt.show()

In [ ]:
generate_text(model_single)

In [ ]:
generate_text(model_double)